# P0: Alohomora!
# Part1:
## Table Of Content

1. What Is Convolution?
2. Software Setup
3. Convolution Methods
    - Part-A.1: Keep Calm And Use `scipy.signal.convolve2d`
    - Part-A.2: Slow And Steady: Applying Convolution With For-Loops
    - Part-A.3: Lets Torch It
4. Grading
5. Report guidelines

## 1. What is Convolution?

Convolution is a fundamental operation in image processing, computer vision, and deep learning. It plays a crucial role in extracting features from images, enabling the detection of edges, textures, and patterns. At its core, convolution involves the application of a filter or kernel to an image, transforming it into a new representation that highlights specific features. Read up on [Convolution](https://en.wikipedia.org/wiki/Convolution) from Wikipedia for more details.

Convolution is a mathematical operation that takes two inputs: an image (a 2D matrix of pixel values) and a kernel (a smaller matrix, often called a filter). The kernel is systematically moved (or convolved) across the image, and at each position, the element-wise product of the overlapping pixels is summed to produce a new pixel value in the output image. This process effectively combines the original image's information with the filter's characteristics, emphasizing certain features such as edges or textures.

#### The Jargon:

- **Kernel (Filter):** A small matrix used in convolution to modify the image. [Common kernels](https://en.wikipedia.org/wiki/Kernel_(image_processing)) include Gaussian, Sobel or Prewitt operators.
- **Stride:** The step size with which the kernel moves across the image. A stride of 1 means the kernel moves one pixel at a time, while a larger stride results in a more significant jump between positions.
- **Padding:** Padding involves adding extra pixels (usually zeros) around the edges of the image to control the size of the output image. Without padding, the output image will be smaller than the input.
- **Convolution vs. Cross-Correlation:** In true convolution, the kernel is flipped before being applied (by element-wise dot product and summation) to the image. In cross-correlation, the kernel is used as-is. While convolution is the traditional mathematical operation, many libraries (including deep learning frameworks) use cross-correlation due to its simplicity and efficiency. In this project, you will be applying **convolution** operation and **not cross-correlation**. 

#### Applications of Convolution:

Convolution is widely used in various fields, particularly in image processing and deep learning:
- **Edge Detection:** Kernels designed to highlight edges can be convolved with an image to reveal its structure and boundaries.
- **Blurring and Smoothing:** Convolution with a uniform kernel can smooth out noise and details, resulting in a blurred image.
- **Feature Extraction:** In deep learning, convolutional layers automatically learn kernels that extract hierarchical features from images, enabling tasks such as object detection, classification, and segmentation.

#### Summary of this project
In this project, you will implement convolution in three different ways. This hands-on approach will deepen your understanding of how convolution functions, implementation issues and its significance in image analysis.

Here is a sample output.

![Minion](./sample/sample.png)


**Filter Specification:**

- Filter stride must be 1.
- The image dimension must not change after filtering operation (use only zeroes for padding).
- Output should be in double datatype. Please cast your input images to double before the filtering operation.
- Use the provided kernel (KERNEL) only.

## 2. Software Setup

Ubuntu is the only supported platform for all the assignments in this course. You are responsible for installing the required packages yourself. We highly recommend using a virtual environment, such as venv or conda, to manage dependencies and maintain a clean development setup.

The outputs from your functions will be saved as image files to assist you in debugging.

The functions you complete will be evaluated using an automated test.py script. To check if your implementations pass the tests, open a terminal in the current folder and run python test.py.

In [2]:
# Imports 
import numpy as np
import torch
from torchvision import transforms
import scipy
import time
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import nbimporter
import unittest

from utils import *

image_pil = Image.open('main.jpg').convert('L')
image = np.array(image_pil)

KERNEL = np.array(
    [
        [1, 0, -1], 
        [2, 0, -2],
        [1, 0, -1]
    ]
)

## 3. Convolution Methods
### Part1.1: Keep Calm And Use `scipy.signal.convolve2d`

Several libraries provide standard implementations of convolution filters for image processing tasks. In this section, you will implement convolution using `scipy.signal.convolve2d`.

Using the convolve2d function from the scipy.signal module, apply a convolution filter to an RGB image below. 

In [3]:
def filter_scipy_convolve2d(img, kernel):
    start =time.time()
    filtered_image = np.zeros_like(img).astype(np.float64)
    
    # Write Your Code Here!
    # Convolve the image with the kernel using scipy.signal.convolve2d
    filtered_image = scipy.signal.convolve2d(img, kernel, mode='same')
    
    print("Elapsed time (s)=", time.time() - start)
    return filtered_image

In [4]:
img = filter_scipy_convolve2d(image, KERNEL)
writeDoubleImage(img, "scipy.jpg")

Elapsed time (s)= 0.6803853511810303


### Part1.2: Slow And Steady: Applying Convolution With For-Loops

Now implement the same functionality using "for" loop. The output image should have the same shape as input. Use zeroes for padding the input image at edges so that the output image has the same shape as input after convolution.

Hint -  You can pad the input matrix with zeros using np.pad. For flipping the kernel, use np.flip()

In [5]:
def filter_numpy_for_loop(img, kernel):
    start = time.time()
    
    filtered_image = np.zeros_like(img).astype(np.float64)

    # Convert img to float to get more accurate results
    img = img.astype(np.float64)
    
    # Convolve the image with the kernel using nested for loops
    for i in range(1, img.shape[0]-1):
        for j in range(1, img.shape[1]-1):
            # find the element-wise product of the flipped kernel and the image patch
            filtered_image[i, j] = np.sum(np.flip(kernel) * img[i-1:i+2, j-1:j+2])

    print("Elapsed time (s)=", time.time() - start)
    return filtered_image

In [6]:
img = filter_numpy_for_loop(image, KERNEL)

writeDoubleImage(img, "numpy_for_loop.jpg")

Elapsed time (s)= 1.2815091609954834


### Part1.3: Lets Torch It
Convolutional filters (conv filters) are small, learnable matrices used in convolutional neural networks (CNNs) to detect patterns such as edges, textures, or shapes in input data. By sliding over the input (e.g., an image), they compute dot products to create feature maps, highlighting specific aspects of the data relevant for tasks like classification or detection.

In the next section, you will use a Conv. layer with hard-coded weights to perform the same task. You must use a [conv2d layer](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html) with hard-coded kernel. Make sure you flip the kernel using np.flip(kernel).copy().

In [7]:
def filter_torch(img, kernel):
    start = time.time()
    print(img.shape)

    # Convert our Kernel to a tensor
    kernel = torch.tensor(np.flip(kernel).copy(),dtype=torch.float64)

    # Create our conv2d layer
    conv = torch.nn.Conv2d(1,1,kernel_size=kernel.shape,stride=1, padding=1,bias=False)

    # Initalize weights with our kernel
    with torch.no_grad():
        conv.weight = torch.nn.Parameter(kernel.unsqueeze(0).unsqueeze(0)) 
    
    # Convert our image to a tensor
    img = torch.tensor(img,dtype=torch.float64).unsqueeze(0)

    # Perform the convolution
    filtered_image = conv(img)
    print(filtered_image.shape)

    print("Elapsed time (s)=", time.time() - start)
    return filtered_image.squeeze(0).squeeze(0).detach().numpy()

In [8]:
img = filter_torch(image, KERNEL)
writeDoubleImage(img, "torch_conv.jpg")

(384, 683)
torch.Size([1, 384, 683])
Elapsed time (s)= 0.047693490982055664


## 4. Grading Rubric for Part1

- Part1.1 25%
- Part1.2 25%
- Part1.3 25%
- Report   25%

## 5. Report Guidelines:

- Include the execution time for all three techniques. If there are differences, explain the reasons behind them.
- The report must include both the expected ground truth (from the test.py code) and the achieved convolved images.
- Clearly describe the filtering operation being performed and explain the role of the kernel. Support your explanation with evidence, using images from different scenarios to illustrate your points.
- What is happening in the kernel you used and why does it work?